In [1]:
import time
import pickle
import os
import json
import h5py
from PIL import Image
import numpy as np
#from utils import *
import tensorflow as tf
from tensorflow import keras
# from keras.models import Model
# from keras.applications.vgg16 import VGG16
from sklearn.linear_model import Ridge
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD

#  Utils

In [2]:
def write_hdf5(arr, outfile):
    """
        Write an numpy array to a file in HDF5 format.
    """
    with h5py.File(outfile, "w", libver='latest') as f:
        f.create_dataset("image", data=arr, dtype=arr.dtype)

def load_hdf5(infile):
    """
        Load a numpy array stored in HDF5 format into a numpy array.
    """
    with h5py.File(infile, "r", libver='latest') as hf:
        return hf["image"][:]


In [3]:
def img_to_array(data_path, desired_size=None):
    """
        Util function for loading RGB image into 3D numpy array.
        Returns array of shape (H, W, C)

        References
        ----------
    - adapted from keras preprocessing/image.py
    """
    img = Image.open(data_path)
    img = img.convert('RGB')
    if desired_size:
        img = img.resize((desired_size[1], desired_size[0]))
    x = np.asarray(img, dtype='float32')
    return x
def preprocess_img(img_path, desired_dims):
    """
        Loads image using img_to_array, expands it to 4D tensor
        of shape (1, H, W, C), preprocesses it for use in the
        VGG16 network and resequeezes it to a 3D tensor.

        References
        ----------
        - adapted from keras preprocessing/image.py
    """
    img = img_to_array(data_path=img_path, desired_size=desired_dims)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg16.preprocess_input(img)
    img = np.squeeze(img)
    return img

def array_to_img(x):
    """
        Util function for converting 4D numpy array to numpy array.

        Returns PIL RGB image.

        References
        ----------
        - adapted from keras preprocessing/image.py
    """
    x = np.asarray(x)
    x += max(-np.min(x), 0)
    x_max = np.max(x)
    if x_max != 0:
        x /= x_max
    x *= 255
    return Image.fromarray(x.astype('uint8'), 'RGB')


# Convert Video to frame

In [6]:
import cv2
vidcap = cv2.VideoCapture('project/drive.mp4')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

...

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [4]:
import pandas as pd

In [5]:
obj = pd.read_json('output/drive.json')
obj.head()

,0,1
0,0.004662,2.022715
1,0.069798,2.040872
2,0.129914,2.062394
3,0.226643,2.076283
4,0.291367,2.077074


In [6]:
print(obj)

               0         1
0       0.004662  2.022715
1       0.069798  2.040872
2       0.129914  2.062394
3       0.226643  2.076283
4       0.291367  2.077074
...          ...       ...
8611  732.669017  0.000000
8612  732.783648  0.000000
8613  732.833436  0.000000
8614  732.863809  0.000000
8615  732.952056  0.000000

[8616 rows x 2 columns]


# Preprocessing

In [7]:
out_path = 'output/'
img_path = 'frames/'

# img params
height = 50
width = 50
channels = 3

# create mask for splitting (test will be 10% of data)
train_mask = np.arange(7754)
test_mask = np.arange(7754, 7754+862)

# grab all file names
included_extensions = ['jpg']
file_names = [fn for fn in os.listdir(img_path)
                if any(fn.endswith(ext) for ext in included_extensions)]

# store total number of images
num_imgs = len(file_names)

# initialize img array which will hold frames
X = np.empty((num_imgs, height, width, channels), dtype='float32')

print("\nConverting JPGs to numpy array...")
print("\n")


Converting JPGs to numpy array...




In [8]:
# loop and convert jpg to numpy array
for i in range(num_imgs):
    if i % 1000 == 0:
        print("\tProcessing img {}".format(i))
    filepath = os.path.join(img_path, file_names[i])
    img = preprocess_img(filepath, desired_dims=(height, width))
    X[i] = img

print("\nConverting groundTruth labels to numpy array...")



	Processing img 0
	Processing img 1000
	Processing img 2000
	Processing img 3000
	Processing img 4000
	Processing img 5000
	Processing img 6000
	Processing img 7000
	Processing img 8000

Converting groundTruth labels to numpy array...


NameError: name 'gt_path' is not defined

In [9]:
# this part is for the groundTruth labels
with open(out_path + 'drive.json') as f:
    data = json.load(f)

# convert to numpy array
data = np.asarray(data)

# extract speed
y = data[:, 1]
print(y)

[2.02271467 2.04087188 2.06239438 ... 0.         0.         0.        ]


In [10]:
# shuffle
print("\nShuffling the data...")
X, y = shuffle(X, y, random_state=42)

# split into train and test
print("\nSplitting X into train and test...")
X_train = X[train_mask]
X_test = X[test_mask]

print("\nWriting X_train as HDF5...")
write_hdf5(X_train, out_path + "X_train_50.hdf5")

print("\nWriting X_test as HDF5...")
write_hdf5(X_test, out_path + "X_test_50.hdf5")

# split into train and test
print("\nSplitting y into train and test...")
y_train = y[train_mask]
y_test = y[test_mask]

# add dimension for scikit-learn api
y_train = np.expand_dims(y_train, axis=1)
y_test = np.expand_dims(y_test, axis=1)
print("\n\ty_train shape: {}".format(y_train.shape))
print("\n\ty_test shape: {}".format(y_test.shape))

print("\nWriting y_train as HDF5...")
write_hdf5(y_train, out_path + "y_train_50.hdf5")

print("\nWriting y_test as HDF5...")
write_hdf5(y_test, out_path + "y_test_50.hdf5")

print("\nDone!")


Shuffling the data...

Splitting X into train and test...

Writing X_train as HDF5...

Writing X_test as HDF5...

Splitting y into train and test...

	y_train shape: (7754, 1)

	y_test shape: (862, 1)

Writing y_train as HDF5...

Writing y_test as HDF5...

Done!


# Model Train & Run

In [16]:
def load_data(path):
    # load training data
    X_train = load_hdf5(path + "X_train_50.hdf5")
    y_train = load_hdf5(path + "y_train_50.hdf5")

    # load test data
    X_test = load_hdf5(path + "X_test_50.hdf5")
    y_test = load_hdf5(path + "y_test_50.hdf5")

    return X_train, y_train, X_test, y_test


In [20]:
def get_vgg_features(dump_path):
    """
    Computes the features of the frame images from
    the 'block2_pool' layer of the VGG16 network.

    Writes them as HDF5 when done.
    """

    # load the data
    X_train, y_train, X_test, y_test = load_data(dump_path)

    # dimension sanity check
    print("Train samples: {}".format(X_train.shape))
    print("Test samples: {}".format(X_test.shape))

    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

    # # print layer names
    # for i, layer in enumerate(model.layers):
    #     print(i, layer.name)

    model = tf.keras.Model(inputs=base_model.input, outputs=base_model.get_layer('block2_pool').output)

    print("Computing train features...")
    train_features = model.predict(X_train)
    print("Train Features Shape: {}".format(train_features.shape))
    train_features = np.reshape(train_features, [train_features.shape[0], -1])

    print("Computing test features...")
    test_features = model.predict(X_test)
    print("Test Features Shape: {}".format(test_features.shape))
    test_features = np.reshape(test_features, [test_features.shape[0], -1])

    print("writing as HDF5...")
    write_hdf5(train_features, dump_path + "train_features_200.hdf5")
    write_hdf5(test_features, dump_path + "test_features_200.hdf5")

    return


In [24]:
def main():
    # load the data
    dump_path = 'output/'
    X_train, y_train, X_test, y_test = load_data(dump_path)

    # compute vgg features and write to hdf5
    get_vgg_features(dump_path)

    print("Loading features from HDF5 files...")
    train_features = load_hdf5(dump_path + "train_features_200.hdf5")
    test_features = load_hdf5(dump_path + "test_features_200.hdf5")
    y_train = load_hdf5(dump_path + "y_train_50.hdf5")
    y_test = load_hdf5(dump_path + "y_test_50.hdf5")

    print("Training Linear Regression model...")
    clf = Ridge(alpha=5.0)
    tic = time.time()
    clf.fit(train_features, y_train)
    toc = time.time()
    print("Time elapsed: {} seconds".format(toc - tic))
    print("Predicting...")
    predictions = clf.predict(test_features)
    mse = np.mean((predictions - y_test) ** 2)
    print("MSE: {}".format(mse))
    # dump model
    print("Dumping model...")
    pickle.dump(clf, open(dump_path + "50_clf_all.sav", "wb"))

    clf = pickle.load(open(dump_path + "50_clf_all.sav", "rb"))
    print("Predicting...")
    predictions = clf.predict(test_features)
    mse = np.mean((predictions - y_test) ** 2)
    print("MSE: {}".format(mse))

In [ ]:
# print("Visualizing TSNE embedding...")
    # tic = time.time()
    # X_reduced = TruncatedSVD(n_components=50, random_state=0).fit_transform(train_features)
    # toc = time.time()
    # print("SVD time: {} seconds".format(toc - tic))
    # model = TSNE(n_components=2, perplexity=40, verbose=2, random_state=0)
    # tic = time.time()
    # Y = model.fit_transform(X_reduced)
    # toc = time.time()
    # print("TSNE time: {} seconds".format(toc - tic))
    # write_hdf5(Y, dump_path + "tsne_50.hdf5")

    # # visualize
    # fig = plt.figure(figsize=(10, 10))
    # ax = plt.axes(frameon=False)
    # plt.setp(ax, xticks=(), yticks=())
    # plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9, wspace=0.0, hspace=0.0)
    # plt.scatter(Y[:, 0], Y[:, 1], c=y_train, cmap=plt.cm.Spectral)
    # plt.savefig('tsne.png')

In [ ]:
main()

Train samples: (7754, 50, 50, 3)
Test samples: (862, 50, 50, 3)
Computing train features...
Train Features Shape: (7754, 12, 12, 128)
Computing test features...
